In [5]:
import json
import geojson

In [18]:
class BasicQSP:
    def __init__(self):
        self.id = 'dispatch-Gui'
        self.type = 'statePlan'
        self.features = []
        self.eventList = []
        self.stateVars = []
        self.startEvent = None
        self.goalEpisodes = []
        self.version='0.0-0'
        self.qsp_json = ''

    def setVariable(self, varName, varValue, agent="auv"):
        var = {'$id': '{}.{}'.format(agent, varName),
                '$type': 'stateVariable',
                'timeDomain': {'$type': 'realDomain',
                               'ranges': [{'bounds': [0, 'Infinity'],
                                           'maxClosed': False,
                                           'minClosed': True}]},
                'range': {'$type': 'realDomain',
                          'ranges': [{'bounds': ['-Infinity', 'Infinity'],
                                      'maxClosed': True,
                                      'minClosed': True}]},
                '$annotations': {'rmpl/initialValue': varValue,
                                 'odo.mtk/trueId': {'$type': 'symbol',
                                                    'symbolName': '{}.{}'.format(agent, varName)}}}
        self.stateVars.append(var)

    def initVariables(self, initloc=(0,0), agent="auv"):
        self.setVariable('x-loc', initloc[0], agent)
        self.setVariable('y-loc', initloc[1], agent)

    def addEpisode(self, episode):
        self.eventList.append(episode.start.event)
        self.eventList.append(episode.end.event)
        if self.startEvent == None:
            self.startEvent = episode.start.id
        else:
            # TODO: Add connector episode
            pass
        self.goalEpisodes.append(episode.episode)
        return True # successful add

    def populate(self):
        self.qsp = self.inner_populate()
        self.qsp_json = json.dumps(self.qsp)
        return self.qsp

    def inner_populate(self):
        if self.startEvent == None:
            print("[WARNING]: There is no starting event - QSP is incomplete!!!!")
        return {'$id': self.id,
                '$type': self.type,
                '$features': self.features,
                'stateSpace': {'events': self.eventList,
                               'stateVariables': self.stateVars},
                'startEvent': self.startEvent,
                'goalEpisodes': self.goalEpisodes,
                '$version': self.version}

    def __repr__(self):
        return self.qsp_json

In [3]:
class BasicPosiEpisode:
    def __init__(self, epName, cloc = (0.0,0.0), agent="auv"):
        self.start = BasicEvent(name="{}-s".format(epName))
        self.end = BasicEvent(name="{}-f".format(epName))
        self.duration = {'$type': 'simpleDuration', 'lowerBound': 1, 'upperBound': 1}
        self.constraints = [{'$type': 'stateConstraint',
                             'expression': {'$type': 'equalApplication',
                                            'left': {'$type': 'stateVarApplication',
                                                     'stateVar': '{}.x-loc'.format(agent)},
                                            'right': cloc[0]}},
                            {'$type': 'stateConstraint',
                             'expression': {'$type': 'equalApplication',
                                            'left': {'$type': 'stateVarApplication',
                                                     'stateVar': '{}.y-loc'.format(agent)},
                                            'right': cloc[1]}}]
        self.annotations = {'basicGui': True, 'magellan': False}
        self.episode = self.populate()
        self.episode_json = json.dumps(self.episode)

    def populate(self):
        return {'$type': 'episode',
                'startEvent': self.start.id,
                'endEvent': self.end.id,
                'duration': self.duration,
                'endConstraints': self.constraints,
                '$annotations': self.annotations}

    def __repr__(self):
        return self.episode_json

In [4]:
class BasicEvent:
    def __init__(self, name='ev-1-s', style='event'):
        self.id = name
        self.style = style
        self.domain = {'$type': 'realDomain',
                       'value': self.set_domain_value()}
        self.annotations = {'basicGui': True, 'magellan': False}
        self.event = self.populate()
        self.event_json = json.dumps(self.event)

    def populate(self, use_new=False, name='ev-1-s', style='event'):
        if use_new:
            self.id = name
            self.style = style
            self.domain['value'] = self.set_domain_value()
        return {'$id': self.id,
                '$type': self.style,
                'domain': self.domain,
                '$annotations': self.annotations}

    def set_domain_value(self):
        return 0 if self.id[-1]=='s' else 1

    def __repr__(self):
        return self.event_json

BasicEvent()

{"$id": "ev-1-s", "$type": "event", "domain": {"$type": "realDomain", "value": 0}, "$annotations": {"basicGui": true, "magellan": false}}

In [5]:
import folium as fl
from folium.plugins import Draw
import geojson

m = fl.Map()

Draw(

    export=True,

    filename="my_data.geojson",

    position="topleft",

    draw_options={"polyline": {"allowIntersection": False}},

    edit_options={"poly": {"allowIntersection": False}},

).add_to(m)

In [6]:
m

In [6]:
class GeoParse:
    def __init__(self, geojson_file_path="my_plan.geojson"):
        with open(geojson_file_path) as gjf:
            wpt_data = geojson.load(gjf)
        self.wpts = [dat['geometry']['coordinates'] for dat in wpt_data['features']]
        self.construct_qsp()
        self.write_mi_string()
        self.write_to_file()

    def construct_qsp(self):
        self.qsp = BasicQSP()
        self.qsp.initVariables(tuple(self.wpts[0][::-1]))
        for i in range(1,len(self.wpts)):
            self.qsp.addEpisode(BasicPosiEpisode("ep-{}".format(i), cloc=self.wpts[i][::-1]))
        self.qsp.populate()

    def write_mi_string(self):
        self.mi_string = self.stateplan2ma(self.qsp.qsp_json)

    def write_to_file(self):
        return self.mi_string

    def get_state_var_val(self, state_plan, var_name):
        vars = state_plan["stateSpace"]["stateVariables"]
        for v in vars:
            if v["$id"] == var_name:
                return v["$annotations"]["rmpl/initialValue"]
        return None

    def stateplan2ma(self, state_plan_json):
        state_plan = json.loads(state_plan_json)
        count = 1
        exists = True
        waypoints = []
        init_long = self.get_state_var_val(state_plan, "auv.x-loc")
        init_lat = self.get_state_var_val(state_plan, "auv.y-loc")
        waypoints.append((init_long, init_lat))
        for ep in state_plan["goalEpisodes"]:
            constr = ep["endConstraints"]
            long = constr[0]['expression']['right']
            lat = constr[1]['expression']['right']
            waypoints.append((long,lat))

        return self.waypoints2file(waypoints)

    def waypoints2file(self, waypoints, start_time=0, list_stop_when=7, init_point=-2):
        """
        Given a list of waypoints, generate the string corresponding to the gotolist command
        :param waypoints: list of waypoint tuples
        :param start_time: when to start the list (0-immediatley, 1-stack idle, 2-heading idle)
        :param list_stop_when: When to go to next way point
        :param init_point: which waypoint to start the closest to
        :return: a string matching mi file syntax
        """

        out = "behavior_name=goto_list\n"
        out += "<start:b_arg>\n"

        out += "\tb_arg: start_when(enum) {}\n".format(start_time)
        out += "\tb_arg: list_stop_when(enum) {}\n".format(list_stop_when)
        out += "\tb_arg: init_wpt(enum) {}\n".format(init_point)
        out += "\tb_arg: num_waypoints(nodim) {}\n".format(len(waypoints))
        out += "<end:b_arg>\n<start:waypoints>\n"
        for w in waypoints:
            out+="{}    {}\n".format(w[1],w[0])
        out+="<end:waypoints>"
        return out

In [3]:
print(GeoParser("trial-data/trial_plan.geojson").mi_string)

behavior_name=goto_list
<start:b_arg>
	b_arg: start_when(enum) 0
	b_arg: list_stop_when(enum) 7
	b_arg: init_wpt(enum) -2
	b_arg: num_waypoints(nodim) 3
<end:b_arg>
<start:waypoints>
-70.552386    41.524001
-70.464449    41.522973
-70.62933    41.508577
<end:waypoints>


In [2]:
from utils.episodes import BasicPosiEpisode
from utils.events import BasicEvent
from utils.qsps import BasicQSP

In [1]:
from geoparse import GeoParser